<a href="https://colab.research.google.com/github/junisatria20/DeepLearning/blob/main/MidtermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Juni Satria Siregar

NPM : 2306288906

In [ ]:
# Clone YOLOv5 dari GitHub
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Install dependencies untuk YOLOv5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 9.97 MiB/s, done.
Resolving deltas: 100% (11694/11694), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.5/877.5 kB 8.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Deteksi objek pada video yang diunggah
!python detect.py --source /content/drive/MyDrive/VideoHewan.mp4 --weights yolov5s.pt --conf 0.4


detect: weights=['yolov5s.pt'], source=/content/drive/MyDrive/VideoHewan.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-378-g2f74455a Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
video 1/1 (1/2174) /content/drive/MyDrive/VideoHewan.mp4: 640x384 (no detections), 678.7ms
video 1/1 (2/2174) /content/drive/MyDrive/VideoHewan.mp4: 640x384 1 elephant, 1 giraffe, 865.3ms
video 1/1 (3/2174) /content/drive/MyDrive/VideoHewan.mp4: 640x384 1 elephant, 1 giraffe, 393.9ms
video 1/1 (4/2174) /content/drive/MyDrive/VideoHewan.m

In [ ]:
import torch
from pathlib import Path
import cv2

# Load model YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Lokasi file video
video_path = '/content/drive/MyDrive/VideoHewan.mp4'

# Buka video
cap = cv2.VideoCapture(video_path)

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Proses frame dengan YOLOv5
    results = model(frame)

    # Ekstrak bounding boxes dan class labels
    detections = results.xyxy[0].cpu().numpy()  # Bounding boxes and class labels

    for det in detections:
        xmin, ymin, xmax, ymax, confidence, class_id = det
        label = model.names[int(class_id)]  # Label nama objek

        # Tampilkan hasil deteksi pada frame (bounding box dan label)
        cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
        cv2.putText(frame, f'{label} {confidence:.2f}', (int(xmin), int(ymin)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Simpan atau tampilkan frame hasil deteksi (misalnya, dengan cv2.imshow)
    frame_count += 1

cap.release()
cv2.destroyAllWindows()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-24 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/comm

In [ ]:
import torch
import cv2
import pandas as pd

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Path to video (pastikan path-nya benar)
video_path = '/content/drive/MyDrive/VideoHewan.mp4'

# Fungsi untuk menormalkan bounding boxes
def normalize_bbox(detections, frame_width, frame_height):
    """Normalisasi bounding boxes menjadi rasio relatif terhadap ukuran frame."""
    normalized = []
    for det in detections:
        xmin, ymin, xmax, ymax, confidence, class_id = det
        # Normalisasi koordinat berdasarkan ukuran frame
        xmin /= frame_width
        xmax /= frame_width
        ymin /= frame_height
        ymax /= frame_height
        normalized.append([xmin, ymin, xmax, ymax, confidence, class_id])
    return normalized

# Fungsi untuk mengekstrak hasil deteksi ke format yang terstruktur
def extract_detections(detections, frame_width, frame_height, frame_number):
    """Ekstrak hasil deteksi ke dalam format yang terstruktur (normalisasi bounding box)."""
    normalized_detections = normalize_bbox(detections, frame_width, frame_height)
    data = []
    for det in normalized_detections:
        xmin, ymin, xmax, ymax, confidence, class_id = det
        label = model.names[int(class_id)]  # Nama label objek dari YOLO
        # Simpan hasil dalam format terstruktur
        data.append({
            'frame': frame_number,
            'label': label,
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax,
            'confidence': confidence
        })
    return data

# Buka video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Tidak dapat membuka video di path {video_path}.")
else:
    # Dapatkan ukuran frame
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Simpan hasil dalam list untuk analisis lebih lanjut
    detection_results = []

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Proses setiap frame dengan YOLOv5
        results = model(frame)
        detections = results.xyxy[0].cpu().numpy()  # Bounding boxes dan class labels

        # Ekstrak hasil deteksi untuk setiap frame
        frame_detections = extract_detections(detections, frame_width, frame_height, frame_count)
        detection_results.extend(frame_detections)

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

    # Konversi hasil deteksi menjadi DataFrame untuk dianalisis lebih lanjut
    df_results = pd.DataFrame(detection_results)
    print(df_results.head())  # Tampilkan beberapa hasil pertama


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-24 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/comm

   frame      label      xmin      ymin      xmax      ymax  confidence
0      1  surfboard  0.455890  0.037348  1.000000  0.553518    0.357085
1      2  surfboard  0.460268  0.037452  1.000000  0.582428    0.346393
2      4     person  0.002664  0.008133  0.164043  0.496206    0.256750
3      5  surfboard  0.431964  0.037835  1.000000  0.545499    0.365744
4      6  surfboard  0.445408  0.034968  1.000000  0.542050    0.404147


In [ ]:
import torch
import cv2
from collections import OrderedDict
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Fungsi untuk menghitung centroid dari bounding box
def compute_centroid(xmin, ymin, xmax, ymax):
    """Menghitung centroid dari bounding box."""
    return (int((xmin + xmax) / 2), int((ymin + ymax) / 2))

# Simple Object Tracker dengan centroid tracking
class SimpleTracker:
    def __init__(self):
        self.next_object_id = 0  # ID objek yang akan diberikan
        self.objects = OrderedDict()  # Dictionary untuk menyimpan objek yang dilacak
        self.disappeared = OrderedDict()  # Melacak berapa lama objek telah hilang
        self.max_disappeared = 10  # Batas frame di mana objek dianggap hilang

    def register(self, centroid):
        """Mendaftarkan objek baru dengan centroid-nya."""
        self.objects[self.next_object_id] = centroid
        self.disappeared[self.next_object_id] = 0
        self.next_object_id += 1

    def deregister(self, object_id):
        """Menghapus objek dari pelacakan."""
        del self.objects[object_id]
        del self.disappeared[object_id]

    def update(self, input_centroids):
        """Meng-update status objek yang dilacak berdasarkan input centroid baru."""
        if len(input_centroids) == 0:
            # Tingkatkan hitungan "disappeared" untuk setiap objek yang ada
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                # Hapus objek yang sudah terlalu lama hilang
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects

        # Jika tidak ada objek yang sedang dilacak, daftar semua input centroid baru
        if len(self.objects) == 0:
            for centroid in input_centroids:
                self.register(centroid)

        # Update pelacakan berdasarkan centroid yang terdekat
        object_ids = list(self.objects.keys())
        object_centroids = list(self.objects.values())

        # Compute the distance between each pair of object centroids and input centroids
        D = np.linalg.norm(np.array(object_centroids)[:, np.newaxis] - input_centroids, axis=2)

        # Temukan jarak terdekat antara objek yang ada dan input centroid baru
        rows = D.min(axis=1).argsort()
        cols = D.argmin(axis=1)[rows]

        # Lakukan assignment berdasarkan jarak terdekat
        used_rows, used_cols = set(), set()
        for (row, col) in zip(rows, cols):
            if row in used_rows or col in used_cols:
                continue
            object_id = object_ids[row]
            self.objects[object_id] = input_centroids[col]
            self.disappeared[object_id] = 0
            used_rows.add(row)
            used_cols.add(col)

        # Daftarkan centroid yang baru
        unused_cols = set(range(D.shape[1])) - used_cols
        for col in unused_cols:
            self.register(input_centroids[col])

        return self.objects

# Lokasi video
video_path = '/content/drive/MyDrive/VideoHewan.mp4'

# Buka video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Tidak dapat membuka video di path {video_path}.")
else:
    # Inisialisasi pelacak objek
    tracker = SimpleTracker()

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Proses setiap frame dengan YOLOv5
        results = model(frame)
        detections = results.xyxy[0].cpu().numpy()  # Bounding boxes dan class labels

        # Ekstrak centroid dari bounding boxes
        input_centroids = []
        for det in detections:
            xmin, ymin, xmax, ymax, confidence, class_id = det
            centroid = compute_centroid(xmin, ymin, xmax, ymax)
            input_centroids.append(centroid)

        # Update pelacak dengan centroid baru
        objects = tracker.update(np.array(input_centroids))

        # Tampilkan hasil pelacakan
        for (object_id, centroid) in objects.items():
            text = f"ID {object_id}"
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(frame, centroid, 4, (0, 255, 0), -1)

        # Simpan atau tampilkan frame hasil pelacakan
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-24 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/comm

In [ ]:
import torch
import cv2
from collections import OrderedDict
import numpy as np
from google.colab.patches import cv2_imshow  # Menggunakan cv2_imshow untuk Google Colab

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Function to compute centroid from bounding box
def compute_centroid(xmin, ymin, xmax, ymax):
    """Compute centroid of the bounding box."""
    return (int((xmin + xmax) / 2), int((ymin + ymax) / 2))

# Simple Object Tracker with centroid tracking
class SimpleTracker:
    def __init__(self):
        self.next_object_id = 0  # ID for the next object
        self.objects = OrderedDict()  # Dictionary to store tracked objects
        self.disappeared = OrderedDict()  # Track how long each object has been missing
        self.max_disappeared = 10  # Number of frames an object can be missing before deregistration

    def register(self, centroid):
        """Register a new object with its centroid."""
        self.objects[self.next_object_id] = centroid
        self.disappeared[self.next_object_id] = 0
        self.next_object_id += 1

    def deregister(self, object_id):
        """Deregister an object from tracking."""
        del self.objects[object_id]
        del self.disappeared[object_id]

    def update(self, input_centroids):
        """Update the tracking status of objects based on new centroids."""
        if len(input_centroids) == 0:
            # Increase disappearance count for existing objects
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects

        # If no tracked objects, register all new centroids
        if len(self.objects) == 0:
            for centroid in input_centroids:
                self.register(centroid)

        # Update tracking based on nearest centroids
        object_ids = list(self.objects.keys())
        object_centroids = list(self.objects.values())

        # Compute distance between object centroids and input centroids
        D = np.linalg.norm(np.array(object_centroids)[:, np.newaxis] - input_centroids, axis=2)

        # Find nearest centroid for existing objects
        rows = D.min(axis=1).argsort()
        cols = D.argmin(axis=1)[rows]

        used_rows, used_cols = set(), set()
        for (row, col) in zip(rows, cols):
            if row in used_rows or col in used_cols:
                continue
            object_id = object_ids[row]
            self.objects[object_id] = input_centroids[col]
            self.disappeared[object_id] = 0
            used_rows.add(row)
            used_cols.add(col)

        # Register new centroids that weren't matched
        unused_cols = set(range(D.shape[1])) - used_cols
        for col in unused_cols:
            self.register(input_centroids[col])

        return self.objects

# Path to video
video_path = '/content/drive/MyDrive/VideoHewan.mp4'

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Cannot open video at path {video_path}.")
else:
    # Initialize object tracker
    tracker = SimpleTracker()

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process each frame with YOLOv5
        results = model(frame)
        detections = results.xyxy[0].cpu().numpy()  # Bounding boxes and class labels

        # Extract centroids from bounding boxes
        input_centroids = []
        for det in detections:
            xmin, ymin, xmax, ymax, confidence, class_id = det
            centroid = compute_centroid(xmin, ymin, xmax, ymax)
            input_centroids.append(centroid)

        # Update tracker with new centroids
        objects = tracker.update(np.array(input_centroids))

        # Display tracking results
        for (object_id, centroid) in objects.items():
            text = f"ID {object_id}"
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(frame, centroid, 4, (0, 255, 0), -1)

        # Show the frame with tracking results using cv2_imshow
        cv2_imshow(frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
import torch
import cv2
from collections import OrderedDict
import numpy as np

# Enable GPU acceleration and use FP16 for faster inference
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to(device)
model.half()  # Use half precision (FP16) for faster inference

# Function to compute centroid from bounding box
def compute_centroid(xmin, ymin, xmax, ymax):
    """Compute centroid of the bounding box."""
    return (int((xmin + xmax) / 2), int((ymin + ymax) / 2))

# Simple Object Tracker with centroid tracking
class SimpleTracker:
    def __init__(self):
        self.next_object_id = 0  # ID for the next object
        self.objects = OrderedDict()  # Dictionary to store tracked objects
        self.disappeared = OrderedDict()  # Track how long each object has been missing
        self.max_disappeared = 10  # Number of frames an object can be missing before deregistration

    def register(self, centroid):
        """Register a new object with its centroid."""
        self.objects[self.next_object_id] = centroid
        self.disappeared[self.next_object_id] = 0
        self.next_object_id += 1

    def deregister(self, object_id):
        """Deregister an object from tracking."""
        del self.objects[object_id]
        del self.disappeared[object_id]

    def update(self, input_centroids):
        """Update the tracking status of objects based on new centroids."""
        if len(input_centroids) == 0:
            # Increase disappearance count for existing objects
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects

        # If no tracked objects, register all new centroids
        if len(self.objects) == 0:
            for centroid in input_centroids:
                self.register(centroid)

        # Update tracking based on nearest centroids
        object_ids = list(self.objects.keys())
        object_centroids = list(self.objects.values())

        # Compute distance between object centroids and input centroids
        D = np.linalg.norm(np.array(object_centroids)[:, np.newaxis] - input_centroids, axis=2)

        # Find nearest centroid for existing objects
        rows = D.min(axis=1).argsort()
        cols = D.argmin(axis=1)[rows]

        used_rows, used_cols = set(), set()
        for (row, col) in zip(rows, cols):
            if row in used_rows or col in used_cols:
                continue
            object_id = object_ids[row]
            self.objects[object_id] = input_centroids[col]
            self.disappeared[object_id] = 0
            used_rows.add(row)
            used_cols.add(col)

        # Register new centroids that weren't matched
        unused_cols = set(range(D.shape[1])) - used_cols
        for col in unused_cols:
            self.register(input_centroids[col])

        return self.objects

# Path to video
video_path = '/content/drive/MyDrive/Dataset/VideoHewan.mp4'

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Cannot open video at path {video_path}.")
else:
    # Initialize object tracker
    tracker = SimpleTracker()

    frame_count = 0
    frame_skip = 5  # Skip 5 frames for every processed frame (to increase speed)
    input_size = 640  # Input resolution (640x640 for faster inference)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames to speed up processing
        if frame_count % frame_skip != 0:
            frame_count += 1
            continue

        # Resize frame to the desired input size (640x640 or smaller for faster inference)
        resized_frame = cv2.resize(frame, (input_size, input_size))

        # Convert frame to tensor, normalize, add batch dimension, and transfer to GPU
        frame_tensor = torch.from_numpy(resized_frame).permute(2, 0, 1).unsqueeze(0).to(device).half() / 255.0  # Add batch dimension, normalize

        # Process each frame with YOLOv5
        results = model(frame_tensor)

        # Extract detections
        detections = results[0]  # Accessing the first element directly

        # Debug: Print the shape of detections
        print(f"Detections shape: {detections.shape}")

        # Extract centroids from bounding boxes
        input_centroids = []
        for det in detections:
            if det is None:  # Skip if detection is None
                continue

            # Unpack detection values
            xmin, ymin, xmax, ymax, confidence, class_id = det[:6]  # Get the first 6 values

            # Check if confidence is a single scalar
            if isinstance(confidence, torch.Tensor):
                confidence = confidence.item()  # Convert to scalar
            else:
                confidence = confidence[0]  # Take the first value if it's an array

            if confidence > 0.5:  # Only consider detections with high confidence
                centroid = compute_centroid(xmin.item(), ymin.item(), xmax.item(), ymax.item())  # Convert to scalars
                input_centroids.append(centroid)

        # Update tracker with new centroids
        objects = tracker.update(np.array(input_centroids))

        # Display tracking results
        for (object_id, centroid) in objects.items():
            text = f"ID {object_id}"
            cv2.putText(resized_frame, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(resized_frame, centroid, 4, (0, 255, 0), -1)

        # Use cv2_imshow() to display frame in Google Colab (as cv2.imshow is disabled)
        from google.colab.patches import cv2_imshow
        cv2_imshow(resized_frame)

        # Press 'q' to exit (can be removed in Colab since we don’t use interactive windows)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()